# Pipeline ELT

## Extração e Carregamento


Em ELT/cargaRaw.py, os dados são extraídos dos arquivos .CSV da prefeitura que estão na pasta data/

Utilizando a bibilioteca Pandas, extraímos os dados de 2003 até 2005, e de 2018 até 2020. O dados são enviados para o banco de dados um por um. Os dados anteriores e posteriores a 2016 são segregados em duas tabelas diferentes pois valores monetários a partir de 2016 são formatados com vírgula como separador decimal, enquanto os anteriores são formatados com ponto. 


In [ ]:
import os
import sys
import pandas as pd

parent_folder = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
sys.path.append(parent_folder)
from postgres.engine import engine

# Lista dos anos que serão processados na abordagem ELT
anos = (2003, 2004, 2005, 2018, 2019, 2020)

print("=== INICIANDO CARGA ELT - DADOS BRUTOS ===")
print("Carregando dados sem transformação na tabela raw_despesas")

for ano in anos:
    print(f"Processando ano {ano}...")
    caminho = f"data/recife-dados-despesas-{ano}.csv"
    
    # Extração: lê os dados CSV sem nenhuma transformação
    df = pd.read_csv(caminho, sep=';', encoding='utf-8')    
    
    # Load: insere dados brutos diretamente no banco para posterior transformação
    # if_exists="append" adiciona os dados sem sobrescrever registros existentes
    if ano < 2016:
        df.to_sql("raw_despesas_pre_2016", con=engine, if_exists="append", index=False)
    else:
        # Para anos >= 2016, cria uma tabela separada para evitar problemas com vírgula/ponto
        df.to_sql("raw_despesas_pos_2016", con=engine, if_exists="append", index=False)
    
    print(f"  ✓ {len(df)} registros carregados para {ano}")

print("\n=== CARGA ELT CONCLUÍDA ===")
print("Dados brutos disponíveis nas tabelas 'raw_despesas_pre_2016' e 'raw_despesas_pos_2016' para transformação")


## Transformação


Em ELT/transformacaoDados.py os dados são transformados já no banco de dados postgres, utilizando a biblioteca SQLAlchemy para executar comandos SQL diretamente no banco. Os dados passam por 4 etapas principais:

Primeiro, em cada uma das tabelas de dados `raw_despesas_pre_2016` e `raw_despesas_pos_2016`, encontramos todas colunas que são texto e removemos os espaços em branco no início e no fim de cada valor, tornamos todos os caracteres minúsculos e substituímos espaços em branco por underline.

In [ ]:
def clean_text(conn, table):
    # Encontra todas as colunas que são texto
        result = conn.execute(text(f"""
        SELECT column_name 
        FROM information_schema.columns
        WHERE table_name = '{table}'
        AND data_type = 'text';
        """))
        
        columns = [row.column_name for row in result]
        
        # Trim espaços brancos, deixa tudo minúsculo, substitui ' ' por '_'
        for column in columns:
            print(f"Transformação iniciada para {table}.{column}")
            conn.execute(text(f"""
                UPDATE {table}
                SET {column} = REPLACE(LOWER(TRIM({column})), ' ', '_')
                WHERE {column} IS NOT NULL;
            """))
            print(f"Transformação de {table}.{column} finalizada")

Após isso, procuramos por todas as colunas que representam valores monetários em ambas tabelas. Em `raw_despesas_pos_2016`, estas colunas são text, então trocamos ',' por '.', garantimos que não há caracteres não númericos e então convertemos para numeric. Em `raw_despesas_pre_2016`, estas colunas são double precision, então também as convertemos para numeric.

In [ ]:
def valores_monetarios_para_numeric(conn):
    COL_NUMERIC = ('valor_empenhado', 'valor_liquidado', 'valor_pago')
    
    for column in COL_NUMERIC:
        print(f"Transformação iniciada para 'raw_despesas_pos_2016'.{column}")
        
        # Primeiro, limpa os dados removendo caracteres não numéricos (exceto ponto e vírgula)
        # e padroniza o separador decimal para ponto
        conn.execute(text(f"""
            UPDATE raw_despesas_pos_2016
            SET {column} = CASE
                WHEN {column} IS NULL OR TRIM(CAST({column} AS TEXT)) = '' THEN NULL
                ELSE CAST(
                    REPLACE(
                        REPLACE(
                            REGEXP_REPLACE(CAST({column} AS TEXT), '[^0-9,.\\-]', '', 'g'),
                            ',', '.'
                        ),
                        '..', '.'
                    ) AS NUMERIC(18,2)
                )
            END;
        """))
                    
        # Depois altera o tipo da coluna para NUMERIC(18,2)
        for table in ('raw_despesas_pre_2016', 'raw_despesas_pos_2016'):
            conn.execute(text(f"""
                ALTER TABLE {table}
                ALTER COLUMN {column} TYPE NUMERIC(18,2)
                USING {column}::NUMERIC(18,2);
            """))
        
            print(f"Transformação de {table}.{column} finalizada")

Em seguida, convertemos a coluna unidade_codigo para string, pois ela foi inicialmente carregada como double precision por conter um ponto mas na verdade é um código de unidade que não deve ser tratado como número. 

In [ ]:
def tratar_unidade_codigo(conn, table):
    conn.execute(text(f"""
            ALTER TABLE {table}
            ALTER COLUMN unidade_codigo TYPE text
            USING unidade_codigo::text;
    """))

Por fim, criamos a tabela `despesas_recife` caso ela já não tenha sido criado pelo processo de ETL, e inserimos os dados transformados de ambas as tabelas `raw_despesas_pre_2016` e `raw_despesas_pos_2016` nela.

In [ ]:
def unificar_tabelas_despesas(conn):
    # Cria a tabela unificada com base na estrutura de uma das tabelas
    print("Criando tabela unificada 'despesas_recife'...")
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS despesas_recife AS
        SELECT * FROM raw_despesas_pre_2016 WHERE 1=0;
    """))
    
    # Insere dados da primeira tabela
    print("Inserindo dados de 'raw_despesas_pre_2016'...")
    conn.execute(text("""
        INSERT INTO despesas_recife
        SELECT * FROM raw_despesas_pre_2016;
    """))
    
    # Insere dados da segunda tabela
    print("Inserindo dados de 'raw_despesas_pos_2016'...")
    conn.execute(text("""
        INSERT INTO despesas_recife
        SELECT * FROM raw_despesas_pos_2016;
    """))
